## MLP PHASE1

In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import scipy.io
import warnings

warnings.filterwarnings("ignore")


x = scipy.io.loadmat('feat_ex_total_normalized.mat')
x_test = scipy.io.loadmat('feat_test.mat')

Y = scipy.io.loadmat('TrainLabel.mat')

X_test = x_test['feat_test'].T
y = Y['TrainLabel'][0]
# y = [0 if x == -1 else x for x in y]


# Define the parameter grid
param_grid = {'solver' :['lbfgs'] ,
    'hidden_layer_sizes': [(10),(15),(20),(25)],
#     , (50, 30), (30, 20), (20, 10),(50,40,30) , (20,20,20),(10,10,10,10)],
    'activation': ['relu', 'tanh', 'logistic','softmax','identity'],
    'max_iter': [3000],
    'random_state': [42],
}

# Create an MLP classifier
mlp = MLPClassifier(learning_rate = 'adaptive')

# Perform GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the cross-validated accuracy
cross_val_accuracy = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
print(f"Cross-validated Accuracy: {cross_val_accuracy.mean()}")


Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': 20, 'max_iter': 3000, 'random_state': 42, 'solver': 'lbfgs'}
Cross-validated Accuracy: 0.6599999999999999


## MLP GA

In [10]:
x = scipy.io.loadmat('feat_ga_final.mat')
Y = scipy.io.loadmat('TrainLabel.mat')

X = x['feat_ga_final'].T
y = Y['TrainLabel'][0]
# y = [0 if x == -1 else x for x in y]


# Define the parameter grid
param_grid = {'solver' :['lbfgs'] ,
    'hidden_layer_sizes': [(10),(15),(20),(25)],
#     , (50, 30), (30, 20), (20, 10),(50,40,30) , (20,20,20),(10,10,10,10)],
    'activation': ['relu', 'tanh', 'logistic','softmax','identity'],
    'max_iter': [3000],
    'random_state': [42],
}

# Create an MLP classifier
mlp = MLPClassifier(learning_rate = 'adaptive')

# Perform GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the cross-validated accuracy
cross_val_accuracy = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
print(f"Cross-validated Accuracy: {cross_val_accuracy.mean()}")


Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': 15, 'max_iter': 3000, 'random_state': 42, 'solver': 'lbfgs'}
Cross-validated Accuracy: 0.6181818181818182


## RBF PHASE1

In [69]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import math

pi_value = math.pi

x = scipy.io.loadmat('feat_ex_normalized.mat')
Y = scipy.io.loadmat('TrainLabel.mat')

X = x['feat_ex_normalized'].T
y = Y['TrainLabel'][0]
y = np.array([0 if x == -1 else x for x in y])


def gaussian_rbf(x, center, sigma):
    return np.exp(-cdist(x, center, 'sqeuclidean') / (2 * sigma**2))

def cosine_rbf(x, center, sigma):
    d = cdist(x, center, 'sqeuclidean')
    print(d)
    if ( d > 2*sigma):
        return 0
    else :
        return (math.cos(pi_value * d) + 1) / 2


n_centers = 30 # Number of RBF centers
center_indices = np.random.choice(X.shape[0], n_centers, replace=False)
rbf_width = 2
def rbf_layer(X, rbf_centers, rbf_width):
    return gaussian_rbf(X, rbf_centers, rbf_width)

def rbfn_predict(X, rbf_centers, rbf_width, weights):
    rbf_outputs = rbf_layer(X, rbf_centers, rbf_width)
    return rbf

n_centers = np.array([10,15])
rbf_width = np.array([0.1,0.5,1,1.5,2,2.5,3,3.5,3.6,3.7,3.8,3.9,4,4.2,4.4,4.5,5])

# Randomly select center indices

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for i in n_centers:
    for j in rbf_width:
# Perform 5-fold cross-validation
        center_indices = np.random.choice(X.shape[0], i, replace=True)
        rbf_centers = X[center_indices]
        s = 0
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # RBF layer for training set
            rbf_outputs_train = rbf_layer(X_train, rbf_centers, j)

            # Linear regression
            lr = LinearRegression()
            lr.fit(rbf_outputs_train, y_train)

            # RBF layer for test set
            rbf_outputs_test = rbf_layer(X_test, rbf_centers, j)

            # Make predictions on the test set
            y_pred = lr.predict(rbf_outputs_test)

            # Evaluate the model
            accuracy = accuracy_score(y_test, (y_pred >= 0.5).astype(int))
            s = s + accuracy
        print("num of centers : ",i)
        print("rbf width : ", j)
        print(f"Accuracy: {(s/5) * 100:.2f}%")

num of centers :  10
rbf width :  0.1
Accuracy: 50.36%
num of centers :  10
rbf width :  0.5
Accuracy: 49.09%
num of centers :  10
rbf width :  1.0
Accuracy: 47.64%
num of centers :  10
rbf width :  1.5
Accuracy: 52.00%
num of centers :  10
rbf width :  2.0
Accuracy: 55.64%
num of centers :  10
rbf width :  2.5
Accuracy: 57.45%
num of centers :  10
rbf width :  3.0
Accuracy: 52.55%
num of centers :  10
rbf width :  3.5
Accuracy: 56.55%
num of centers :  10
rbf width :  3.6
Accuracy: 58.36%
num of centers :  10
rbf width :  3.7
Accuracy: 54.91%
num of centers :  10
rbf width :  3.8
Accuracy: 54.55%
num of centers :  10
rbf width :  3.9
Accuracy: 56.73%
num of centers :  10
rbf width :  4.0
Accuracy: 53.64%
num of centers :  10
rbf width :  4.2
Accuracy: 55.27%
num of centers :  10
rbf width :  4.4
Accuracy: 57.45%
num of centers :  10
rbf width :  4.5
Accuracy: 55.82%
num of centers :  10
rbf width :  5.0
Accuracy: 56.91%
num of centers :  15
rbf width :  0.1
Accuracy: 50.36%
num of cen

## GA RBF

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import math

pi_value = math.pi

x = scipy.io.loadmat('feat_ga_final.mat')
Y = scipy.io.loadmat('TrainLabel.mat')

X = x['feat_ga_final'].T
y = Y['TrainLabel'][0]
y = np.array([0 if x == -1 else x for x in y])


def gaussian_rbf(x, center, sigma):
    return np.exp(-cdist(x, center, 'sqeuclidean') / (2 * sigma**2))

def cosine_rbf(x, center, sigma):
    d = cdist(x, center, 'sqeuclidean')
    print(d)
    if ( d > 2*sigma):
        return 0
    else :
        return (math.cos(pi_value * d) + 1) / 2


n_centers = 30 # Number of RBF centers
center_indices = np.random.choice(X.shape[0], n_centers, replace=False)
rbf_width = 2
def rbf_layer(X, rbf_centers, rbf_width):
    return gaussian_rbf(X, rbf_centers, rbf_width)

def rbfn_predict(X, rbf_centers, rbf_width, weights):
    rbf_outputs = rbf_layer(X, rbf_centers, rbf_width)
    return rbf

n_centers = np.array([10,15])
rbf_width = np.array([0.1,0.5,1,1.5,2,2.5,3,3.5,3.6,3.7,3.8,3.9,4,4.2,4.4,4.5,5])

# Randomly select center indices

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for i in n_centers:
    for j in rbf_width:
# Perform 5-fold cross-validation
        center_indices = np.random.choice(X.shape[0], i, replace=True)
        rbf_centers = X[center_indices]
        s = 0
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # RBF layer for training set
            rbf_outputs_train = rbf_layer(X_train, rbf_centers, j)

            # Linear regression
            lr = LinearRegression()
            lr.fit(rbf_outputs_train, y_train)

            # RBF layer for test set
            rbf_outputs_test = rbf_layer(X_test, rbf_centers, j)

            # Make predictions on the test set
            y_pred = lr.predict(rbf_outputs_test)

            # Evaluate the model
            accuracy = accuracy_score(y_test, (y_pred >= 0.5).astype(int))
            s = s + accuracy
        print("num of centers : ",i)
        print("rbf width : ", j)
        print(f"Accuracy: {(s/5) * 100:.2f}%")

num of centers :  10
rbf width :  0.1
Accuracy: 49.27%
num of centers :  10
rbf width :  0.5
Accuracy: 54.55%
num of centers :  10
rbf width :  1.0
Accuracy: 55.64%
num of centers :  10
rbf width :  1.5
Accuracy: 54.00%
num of centers :  10
rbf width :  2.0
Accuracy: 56.91%
num of centers :  10
rbf width :  2.5
Accuracy: 56.73%
num of centers :  10
rbf width :  3.0
Accuracy: 53.82%
num of centers :  10
rbf width :  3.5
Accuracy: 52.91%
num of centers :  10
rbf width :  3.6
Accuracy: 55.45%
num of centers :  10
rbf width :  3.7
Accuracy: 53.64%
num of centers :  10
rbf width :  3.8
Accuracy: 56.18%
num of centers :  10
rbf width :  3.9
Accuracy: 54.36%
num of centers :  10
rbf width :  4.0
Accuracy: 54.18%
num of centers :  10
rbf width :  4.2
Accuracy: 55.27%
num of centers :  10
rbf width :  4.4
Accuracy: 54.36%
num of centers :  10
rbf width :  4.5
Accuracy: 52.91%
num of centers :  10
rbf width :  5.0
Accuracy: 56.00%
num of centers :  15
rbf width :  0.1
Accuracy: 47.64%
num of cen